In [1]:
import enum
from typing import List, NamedTuple

import numpy as np

In [4]:
class BodyPart(enum.Enum):
  NOSE = 0
  LEFT_EYE = 1
  RIGHT_EYE = 2
  LEFT_EAR = 3
  RIGHT_EAR = 4
  LEFT_SHOULDER = 5
  RIGHT_SHOULDER = 6
  LEFT_ELBOW = 7
  RIGHT_ELBOW = 8
  LEFT_WRIST = 9
  RIGHT_WRIST = 10
  LEFT_HIP = 11
  RIGHT_HIP = 12
  LEFT_KNEE = 13
  RIGHT_KNEE = 14
  LEFT_ANKLE = 15
  RIGHT_ANKLE = 16


In [5]:
class Point(NamedTuple):
  x: float
  y: float


class Rectangle(NamedTuple):
  start_point: Point
  end_point: Point


class KeyPoint(NamedTuple):
  body_part: BodyPart
  coordinate: Point
  score: float

In [6]:
class Person(NamedTuple):

  keypoints: List[KeyPoint]
  bounding_box: Rectangle
  score: float
  id: int = None


def person_from_keypoints_with_scores(
    keypoints_with_scores: np.ndarray,
    image_height: float,
    image_width: float,
    keypoint_score_threshold: float = 0.1) -> Person:
 

  kpts_x = keypoints_with_scores[:, 1]
  kpts_y = keypoints_with_scores[:, 0]
  scores = keypoints_with_scores[:, 2]
  keypoints = []
  for i in range(scores.shape[0]):
    keypoints.append(
        KeyPoint(
            BodyPart(i),
            Point(int(kpts_x[i] * image_width), int(kpts_y[i] * image_height)),
            scores[i]))

  start_point = Point(
      int(np.amin(kpts_x) * image_width), int(np.amin(kpts_y) * image_height))
  end_point = Point(
      int(np.amax(kpts_x) * image_width), int(np.amax(kpts_y) * image_height))
  bounding_box = Rectangle(start_point, end_point)

  scores_above_threshold = list(
      filter(lambda x: x > keypoint_score_threshold, scores))
  person_score = np.average(scores_above_threshold)

  return Person(keypoints, bounding_box, person_score)


In [7]:
class Category(NamedTuple):
  label: str
  score: float

In [8]:
import os
from typing import Dict, List

import cv2
from data import BodyPart
from data import Person
from data import person_from_keypoints_with_scores
import numpy as np


In [9]:
try:
  from tflite_runtime.interpreter import Interpreter
except ImportError:
  import tensorflow as tf
  Interpreter = tf.lite.Interpreter

In [10]:
class Movenet(object):
  _MIN_CROP_KEYPOINT_SCORE = 0.2
  _TORSO_EXPANSION_RATIO = 1.9
  _BODY_EXPANSION_RATIO = 1.2

  def __init__(self, model_name: str) -> None:
    _, ext = os.path.splitext(model_name)
    if not ext:
      model_name += '.tflite'

    interpreter = Interpreter(model_path=model_name, num_threads=4)
    interpreter.allocate_tensors()

    self._input_index = interpreter.get_input_details()[0]['index']
    self._output_index = interpreter.get_output_details()[0]['index']

    self._input_height = interpreter.get_input_details()[0]['shape'][1]
    self._input_width = interpreter.get_input_details()[0]['shape'][2]

    self._interpreter = interpreter
    self._crop_region = None

  def init_crop_region(self, image_height: int,
                       image_width: int) -> Dict[(str, float)]:
    if image_width > image_height:
      x_min = 0.0
      box_width = 1.0
      y_min = (image_height / 2 - image_width / 2) / image_height
      box_height = image_width / image_height
    else:
      y_min = 0.0
      box_height = 1.0
      x_min = (image_width / 2 - image_height / 2) / image_width
      box_width = image_height / image_width

    return {
        'y_min': y_min,
        'x_min': x_min,
        'y_max': y_min + box_height,
        'x_max': x_min + box_width,
        'height': box_height,
        'width': box_width
    }

  def _torso_visible(self, keypoints: np.ndarray) -> bool:
    left_hip_score = keypoints[BodyPart.LEFT_HIP.value, 2]
    right_hip_score = keypoints[BodyPart.RIGHT_HIP.value, 2]
    left_shoulder_score = keypoints[BodyPart.LEFT_SHOULDER.value, 2]
    right_shoulder_score = keypoints[BodyPart.RIGHT_SHOULDER.value, 2]

    left_hip_visible = left_hip_score > Movenet._MIN_CROP_KEYPOINT_SCORE
    right_hip_visible = right_hip_score > Movenet._MIN_CROP_KEYPOINT_SCORE
    left_shoulder_visible = left_shoulder_score > Movenet._MIN_CROP_KEYPOINT_SCORE
    right_shoulder_visible = right_shoulder_score > Movenet._MIN_CROP_KEYPOINT_SCORE

    return ((left_hip_visible or right_hip_visible) and
            (left_shoulder_visible or right_shoulder_visible))

  def _determine_torso_and_body_range(self, keypoints: np.ndarray,
                                      target_keypoints: Dict[(str, float)],
                                      center_y: float,
                                      center_x: float) -> List[float]:
    torso_joints = [
        BodyPart.LEFT_SHOULDER, BodyPart.RIGHT_SHOULDER, BodyPart.LEFT_HIP,
        BodyPart.RIGHT_HIP
    ]
    max_torso_yrange = 0.0
    max_torso_xrange = 0.0
    for joint in torso_joints:
      dist_y = abs(center_y - target_keypoints[joint][0])
      dist_x = abs(center_x - target_keypoints[joint][1])
      if dist_y > max_torso_yrange:
        max_torso_yrange = dist_y
      if dist_x > max_torso_xrange:
        max_torso_xrange = dist_x

    max_body_yrange = 0.0
    max_body_xrange = 0.0
    for idx in range(len(BodyPart)):
      if keypoints[BodyPart(idx).value, 2] < Movenet._MIN_CROP_KEYPOINT_SCORE:
        continue
      dist_y = abs(center_y - target_keypoints[joint][0])
      dist_x = abs(center_x - target_keypoints[joint][1])
      if dist_y > max_body_yrange:
        max_body_yrange = dist_y

      if dist_x > max_body_xrange:
        max_body_xrange = dist_x

    return [
        max_torso_yrange, max_torso_xrange, max_body_yrange, max_body_xrange
    ]

  def _determine_crop_region(self, keypoints: np.ndarray, image_height: int,
                             image_width: int) -> Dict[(str, float)]:
    target_keypoints = {}
    for idx in range(len(BodyPart)):
      target_keypoints[BodyPart(idx)] = [
          keypoints[idx, 0] * image_height, keypoints[idx, 1] * image_width
      ]

    if self._torso_visible(keypoints):
      center_y = (target_keypoints[BodyPart.LEFT_HIP][0] +
                  target_keypoints[BodyPart.RIGHT_HIP][0]) / 2
      center_x = (target_keypoints[BodyPart.LEFT_HIP][1] +
                  target_keypoints[BodyPart.RIGHT_HIP][1]) / 2

      (max_torso_yrange, max_torso_xrange, max_body_yrange,
       max_body_xrange) = self._determine_torso_and_body_range(
           keypoints, target_keypoints, center_y, center_x)

      crop_length_half = np.amax([
          max_torso_xrange * Movenet._TORSO_EXPANSION_RATIO,
          max_torso_yrange * Movenet._TORSO_EXPANSION_RATIO,
          max_body_yrange * Movenet._BODY_EXPANSION_RATIO,
          max_body_xrange * Movenet._BODY_EXPANSION_RATIO
      ])

      distances_to_border = np.array(
          [center_x, image_width - center_x, center_y, image_height - center_y])
      crop_length_half = np.amin(
          [crop_length_half, np.amax(distances_to_border)])

      if crop_length_half > max(image_width, image_height) / 2:
        return self.init_crop_region(image_height, image_width)
      else:
        crop_length = crop_length_half * 2
      crop_corner = [center_y - crop_length_half, center_x - crop_length_half]
      return {
          'y_min':
              crop_corner[0] / image_height,
          'x_min':
              crop_corner[1] / image_width,
          'y_max': (crop_corner[0] + crop_length) / image_height,
          'x_max': (crop_corner[1] + crop_length) / image_width,
          'height': (crop_corner[0] + crop_length) / image_height -
                    crop_corner[0] / image_height,
          'width': (crop_corner[1] + crop_length) / image_width -
                   crop_corner[1] / image_width
      }
    else:
      return self.init_crop_region(image_height, image_width)

  def _crop_and_resize(
      self, image: np.ndarray, crop_region: Dict[(str, float)],
      crop_size: (int, int)) -> np.ndarray:
    """Crops and resize the image to prepare for the model input."""
    y_min, x_min, y_max, x_max = [
        crop_region['y_min'], crop_region['x_min'], crop_region['y_max'],
        crop_region['x_max']
    ]

    crop_top = int(0 if y_min < 0 else y_min * image.shape[0])
    crop_bottom = int(image.shape[0] if y_max >= 1 else y_max * image.shape[0])
    crop_left = int(0 if x_min < 0 else x_min * image.shape[1])
    crop_right = int(image.shape[1] if x_max >= 1 else x_max * image.shape[1])

    padding_top = int(0 - y_min * image.shape[0] if y_min < 0 else 0)
    padding_bottom = int((y_max - 1) * image.shape[0] if y_max >= 1 else 0)
    padding_left = int(0 - x_min * image.shape[1] if x_min < 0 else 0)
    padding_right = int((x_max - 1) * image.shape[1] if x_max >= 1 else 0)

    output_image = image[crop_top:crop_bottom, crop_left:crop_right]
    output_image = cv2.copyMakeBorder(output_image, padding_top, padding_bottom,
                                      padding_left, padding_right,
                                      cv2.BORDER_CONSTANT)
    output_image = cv2.resize(output_image, (crop_size[0], crop_size[1]))

    return output_image

  def _run_detector(
      self, image: np.ndarray, crop_region: Dict[(str, float)],
      crop_size: (int, int)) -> np.ndarray:
    

    input_image = self._crop_and_resize(image, crop_region, crop_size=crop_size)
    input_image = input_image.astype(dtype=np.uint8)

    self._interpreter.set_tensor(self._input_index,
                                 np.expand_dims(input_image, axis=0))
    self._interpreter.invoke()

    keypoints_with_scores = self._interpreter.get_tensor(self._output_index)
    keypoints_with_scores = np.squeeze(keypoints_with_scores)

    for idx in range(len(BodyPart)):
      keypoints_with_scores[idx, 0] = crop_region[
          'y_min'] + crop_region['height'] * keypoints_with_scores[idx, 0]
      keypoints_with_scores[idx, 1] = crop_region[
          'x_min'] + crop_region['width'] * keypoints_with_scores[idx, 1]

    return keypoints_with_scores

  def detect(self,
             input_image: np.ndarray,
             reset_crop_region: bool = False) -> Person:

    image_height, image_width, _ = input_image.shape
    if (self._crop_region is None) or reset_crop_region:
    
      self._crop_region = self.init_crop_region(image_height, image_width)

    keypoint_with_scores = self._run_detector(
        input_image,
        self._crop_region,
        crop_size=(self._input_height, self._input_width))
  
    self._crop_region = self._determine_crop_region(keypoint_with_scores,
                                                    image_height, image_width)


    return person_from_keypoints_with_scores(keypoint_with_scores, image_height,
                                             image_width)


In [12]:
import tensorflow as tf
import numpy as np
import pandas as pd 
import os
from movenet import Movenet
import wget
import csv
import tqdm 
from data import BodyPart



In [13]:
if('movenet_thunder.tflite' not in os.listdir()):
    wget.download('https://tfhub.dev/google/lite-model/movenet/singlepose/thunder/tflite/float16/4?lite-format=tflite', 'movenet_thunder.tflite')

movenet = Movenet('movenet_thunder')

def detect(input_tensor, inference_count=3):
    movenet.detect(input_tensor.numpy(), reset_crop_region=True)
    
    for _ in range(inference_count - 1):
        detection = movenet.detect(input_tensor.numpy(), 
                                reset_crop_region=False)
    
    return detection


In [14]:
class Preprocessor(object):

        def __init__(self, images_in_folder,
                    csvs_out_path):
            self._images_in_folder = images_in_folder
            self._csvs_out_path = csvs_out_path
            self._csvs_out_folder_per_class = 'csv_per_pose'
            self._message = []
            
            if(self._csvs_out_folder_per_class not in os.listdir()):
                os.makedirs(self._csvs_out_folder_per_class)
            
            self._pose_class_names = sorted(
                [n for n in os.listdir(images_in_folder)]
            )
    

        
        def process(self, detection_threshold=0.1):
            for pose_class_name in self._pose_class_names:
                images_in_folder = os.path.join(self._images_in_folder, pose_class_name)
                csv_out_path = os.path.join(self._csvs_out_folder_per_class,
                                               pose_class_name + '.csv'
                                           )
                with open(csv_out_path, 'w') as csv_out_file:
                    csv_out_writer = csv.writer(csv_out_file,
                                                delimiter=',',
                                                quoting=csv.QUOTE_MINIMAL
                                               )
                    image_names = sorted(
                        [n for n in os.listdir(images_in_folder)]
                    )
                    valid_image_count = 0
                    for image_name in tqdm.tqdm(image_names):
                        image_path = os.path.join(images_in_folder, image_name)
                        
                        try:
                            image = tf.io.read_file(image_path)
                            image = tf.io.decode_jpeg(image)
                        except:
                            self._message.append('Skipped' + image_path + ' Invalid image')
                            continue
                        
                        if image.shape[2] != 3:
                            self.message.append('Skipped' + image_path + ' Image is not in RGB')
                            continue
                        
                        person = detect(image)
                        
                        min_landmark_score = min([keypoint.score for keypoint in person.keypoints])
                        should_keep_image = min_landmark_score >= detection_threshold
                        if not should_keep_image:
                            self._message.append('Skipped' + image_path + 'Keypoints score are below than threshold')
                            continue
                            
                        valid_image_count += 1
                        
                        pose_landmarks = np.array(
                              [[keypoint.coordinate.x, keypoint.coordinate.y, keypoint.score]
                                for keypoint in person.keypoints],
                                  dtype=np.float32)
                        
                   
                        coord = pose_landmarks.flatten().astype(np.str).tolist()
                        csv_out_writer.writerow([image_name] + coord)
                        
            print(self._message)

            all_landmarks_df = self.all_landmarks_as_dataframe()
            all_landmarks_df.to_csv(self._csvs_out_path, index=False)

        def class_names(self):
            return self.pose_class_names
        
        def all_landmarks_as_dataframe(self):
            total_df = None
            for class_index, class_name in enumerate(self._pose_class_names):
                csv_out_path = os.path.join(self._csvs_out_folder_per_class,
                                               class_name + '.csv'
                                           )
                per_class_df = pd.read_csv(csv_out_path, header=None)
                
                per_class_df['class_no'] = [class_index]*len(per_class_df)
                per_class_df['class_name'] = [class_name]*len(per_class_df)
                
                per_class_df[per_class_df.columns[0]] = class_name + '/' +  per_class_df[per_class_df.columns[0]]
                
                if total_df is None:
                    total_df = per_class_df
                else:
                    total_df = pd.concat([total_df, per_class_df], axis=0)
            
            list_name = [[bodypart.name + '_x', bodypart.name + '_y', 
                  bodypart.name + '_score'] for bodypart in BodyPart]
            
            header_name = []
            for columns_name in list_name:
                header_name += columns_name
            header_name = ['filename'] + header_name
            header_map = { total_df.columns[i]: header_name[i]
                             for i in range(len(header_name))
                         }
            
            total_df.rename(header_map, axis=1, inplace=True)
            
            return total_df



In [20]:
import csv
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split
from data import BodyPart 
import tensorflow as tf
import tensorflowjs as tfjs


In [21]:
tfjs_model_dir = 'model'

In [22]:
def load_csv(csv_path):
    df = pd.read_csv(csv_path)
    df.drop(['filename'],axis=1, inplace=True)
    classes = df.pop('class_name').unique()
    y = df.pop('class_no')
    
    X = df.astype('float64')
    y = keras.utils.to_categorical(y)
    
    return X, y, classes


In [23]:
def get_center_point(landmarks, left_bodypart, right_bodypart):
    left = tf.gather(landmarks, left_bodypart.value, axis=1)
    right = tf.gather(landmarks, right_bodypart.value, axis=1)
    center = left * 0.5 + right * 0.5
    return center


In [24]:
def get_pose_size(landmarks, torso_size_multiplier=2.5):
    hips_center = get_center_point(landmarks, BodyPart.LEFT_HIP, 
                                 BodyPart.RIGHT_HIP)

    shoulders_center = get_center_point(landmarks, BodyPart.LEFT_SHOULDER,
                                      BodyPart.RIGHT_SHOULDER)

    torso_size = tf.linalg.norm(shoulders_center - hips_center)
    pose_center_new = get_center_point(landmarks, BodyPart.LEFT_HIP, 
                                     BodyPart.RIGHT_HIP)
    pose_center_new = tf.expand_dims(pose_center_new, axis=1)
    pose_center_new = tf.broadcast_to(pose_center_new,
                                    [tf.size(landmarks) // (17*2), 17, 2])

    d = tf.gather(landmarks - pose_center_new, 0, axis=0,
                name="dist_to_pose_center")
    max_dist = tf.reduce_max(tf.linalg.norm(d, axis=0))

    pose_size = tf.maximum(torso_size * torso_size_multiplier, max_dist)
    return pose_size



In [25]:
def normalize_pose_landmarks(landmarks):
    pose_center = get_center_point(landmarks, BodyPart.LEFT_HIP, 
                                 BodyPart.RIGHT_HIP)

    pose_center = tf.expand_dims(pose_center, axis=1)
    pose_center = tf.broadcast_to(pose_center, 
                                [tf.size(landmarks) // (17*2), 17, 2])
    landmarks = landmarks - pose_center

    pose_size = get_pose_size(landmarks)
    landmarks /= pose_size
    return landmarks


In [26]:

def landmarks_to_embedding(landmarks_and_scores):
    reshaped_inputs = keras.layers.Reshape((17, 3))(landmarks_and_scores)

    landmarks = normalize_pose_landmarks(reshaped_inputs[:, :, :2])
    embedding = keras.layers.Flatten()(landmarks)
    return embedding

In [27]:

def preprocess_data(X_train):
    processed_X_train = []
    for i in range(X_train.shape[0]):
        embedding = landmarks_to_embedding(tf.reshape(tf.convert_to_tensor(X_train.iloc[i]), (1, 51)))
        processed_X_train.append(tf.reshape(embedding, (34)))
    return tf.convert_to_tensor(processed_X_train)

In [40]:
data=pd.read_csv('train_data.csv')
X, y, class_names = load_csv('train_data.csv')
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15)
X_test, y_test, _ = load_csv('test_data.csv')
# print(data)
data.head()


,filename,NOSE_x,NOSE_y,NOSE_score,LEFT_EYE_x,LEFT_EYE_y,LEFT_EYE_score,RIGHT_EYE_x,RIGHT_EYE_y,RIGHT_EYE_score,...,RIGHT_KNEE_y,RIGHT_KNEE_score,LEFT_ANKLE_x,LEFT_ANKLE_y,LEFT_ANKLE_score,RIGHT_ANKLE_x,RIGHT_ANKLE_y,RIGHT_ANKLE_score,class_no,class_name
0,chair/girl1_chair070.jpg,158.0,91.0,0.486216,159.0,86.0,0.567007,154.0,86.0,0.471732,...,223.0,0.659204,152.0,259.0,0.501737,148.0,262.0,0.536225,0,chair
1,chair/girl1_chair070_flipped.jpg,138.0,91.0,0.602957,143.0,87.0,0.507938,139.0,87.0,0.732790,...,220.0,0.879398,152.0,263.0,0.531063,147.0,259.0,0.663975,0,chair
2,chair/girl1_chair075.jpg,165.0,100.0,0.674503,165.0,95.0,0.771247,161.0,95.0,0.767396,...,226.0,0.892319,155.0,262.0,0.701865,152.0,263.0,0.707526,0,chair
3,chair/girl1_chair075_flipped.jpg,134.0,101.0,0.498701,138.0,96.0,0.598086,134.0,95.0,0.545088,...,224.0,0.837559,150.0,265.0,0.762652,145.0,261.0,0.616944,0,chair
4,chair/girl1_chair076.jpg,165.0,102.0,0.640302,166.0,99.0,0.595476,162.0,98.0,0.702522,...,228.0,0.828628,156.0,263.0,0.630904,152.0,264.0,0.672577,0,chair


In [43]:
chairData=pd.read_csv('csv_per_pose/chair.csv')
chairData.head()

,guy3_chair070.jpg,150.0,101.0,0.6963366,155.0,96.0,0.8117418,145.0,96.0.1,0.7756407,...,0.83040226,136.0,245.0,0.71076775,158.0,284.0,0.77941597,141.0,282.0,0.8566091
0,guy3_chair070_flipped.jpg,151.0,101.0,0.772310,155.0,96.0,0.760465,145.0,96.0,0.824127,...,0.838889,143.0,247.0,0.811840,157.0,283.0,0.652070,143.0,287.0,0.774998
1,guy3_chair071.jpg,151.0,103.0,0.720410,155.0,99.0,0.864220,146.0,98.0,0.802676,...,0.777926,136.0,245.0,0.794746,157.0,286.0,0.792072,141.0,281.0,0.780651
2,guy3_chair071_flipped.jpg,150.0,103.0,0.774112,154.0,98.0,0.591950,145.0,97.0,0.652513,...,0.892715,143.0,249.0,0.910520,157.0,282.0,0.710160,143.0,286.0,0.721379
3,guy3_chair072.jpg,150.0,105.0,0.724822,155.0,100.0,0.843660,146.0,100.0,0.798575,...,0.733317,136.0,245.0,0.868371,157.0,287.0,0.787281,141.0,282.0,0.838647
4,guy3_chair072_flipped.jpg,151.0,105.0,0.778852,154.0,100.0,0.595220,145.0,100.0,0.691916,...,0.814658,144.0,249.0,0.838743,157.0,282.0,0.702754,142.0,287.0,0.685322


In [45]:
chairData.size

8684

In [46]:
chairData.shape

(167, 52)

In [50]:
cobraData=pd.read_csv('csv_per_pose/cobra.csv')
cobraData.head()

,guy3_cobra025.jpg,222.0,136.0,0.5634768,224.0,131.0,0.345609,218.0,131.0.1,0.4799904,...,0.33990294,107.0,115.0,0.50060785,100.0,94.0,0.4822154,84.0,99.0,0.4469337
0,guy3_cobra025_flipped.jpg,78.0,136.0,0.649875,82.0,131.0,0.745002,76.0,131.0,0.602871,...,0.478800,171.0,106.0,0.623344,216.0,98.0,0.522433,191.0,93.0,0.469324
1,guy3_cobra026.jpg,220.0,136.0,0.694263,222.0,130.0,0.341933,216.0,129.0,0.481165,...,0.484170,107.0,115.0,0.429484,100.0,95.0,0.592896,86.0,100.0,0.418760
2,guy3_cobra026_flipped.jpg,80.0,136.0,0.781571,85.0,130.0,0.649159,78.0,130.0,0.582968,...,0.713234,171.0,107.0,0.717584,215.0,99.0,0.828097,195.0,92.0,0.579278
3,guy3_cobra027.jpg,215.0,134.0,0.732783,218.0,128.0,0.410006,211.0,128.0,0.659933,...,0.495599,106.0,116.0,0.419999,103.0,96.0,0.500832,85.0,99.0,0.494760
4,guy3_cobra027_flipped.jpg,83.0,135.0,0.543379,88.0,128.0,0.573018,81.0,129.0,0.582031,...,0.624893,172.0,106.0,0.433061,215.0,98.0,0.745913,197.0,92.0,0.420613


In [51]:
cobraData.size

10036

In [52]:
cobraData.shape

(193, 52)

In [53]:
dogData=pd.read_csv('csv_per_pose/dog.csv')
dogData.head()

,guy3_dog025_flipped.jpg,106.0,167.0,0.4450109,108.0,169.0,0.59044814,102.0,169.0.1,0.66342664,...,0.74954164,170.0,131.0,0.5731268,209.0,149.0,0.51584756,191.0,147.0,0.6566826
0,guy3_dog026_flipped.jpg,111.0,169.0,0.295116,112.0,170.0,0.435926,106.0,171.0,0.436138,...,0.735290,166.0,129.0,0.573630,210.0,146.0,0.844132,187.0,143.0,0.472689
1,guy3_dog027.jpg,185.0,167.0,0.428411,190.0,169.0,0.504204,184.0,169.0,0.526251,...,0.423298,110.0,132.0,0.665687,108.0,142.0,0.489317,92.0,146.0,0.763622
2,guy3_dog027_flipped.jpg,112.0,166.0,0.410704,114.0,168.0,0.426121,107.0,168.0,0.366315,...,0.781043,171.0,124.0,0.751789,207.0,146.0,0.843823,189.0,144.0,0.645273
3,guy3_dog028.jpg,182.0,166.0,0.335354,187.0,168.0,0.285620,180.0,168.0,0.421530,...,0.585860,109.0,129.0,0.603919,109.0,143.0,0.550657,93.0,146.0,0.692290
4,guy3_dog028_flipped.jpg,119.0,164.0,0.420272,120.0,166.0,0.440958,113.0,167.0,0.498104,...,0.700576,169.0,123.0,0.622899,207.0,146.0,0.682254,188.0,145.0,0.606661


In [54]:
dogData.size


8736

In [55]:
dogData.shape

(168, 52)

In [56]:
noPoseData=pd.read_csv('csv_per_pose/no_pose.csv')
noPoseData.head()

,1.jpg,357.0,110.0,0.5465723,374.0,100.0,0.82118744,343.0,94.0,0.70673484,...,0.888729,246.0,643.0,0.81805336,370.0,821.0,0.869285,270.0,822.0,0.74990577
0,1_flipped.jpg,240.0,112.0,0.615055,254.0,94.0,0.747929,222.0,100.0,0.827228,...,0.893729,234.0,640.0,0.749313,324.0,821.0,0.803275,228.0,819.0,0.81642


In [57]:
noPoseData.size

52

In [58]:
noPoseData.shape

(1, 52)

In [60]:
sholData=pd.read_csv('csv_per_pose/shoudler_stand.csv')
sholData.head()

,71.Supported-shoulder-stand-Salamba-Sarvangasana-dec21.jpg,553.0,558.0,0.81534344,569.0,571.0,0.6929302,568.0,568.0.1,0.6108195,...,0.8510359,518.0,245.0,0.7076578,547.0,84.0,0.8295817,548.0,86.0,0.7576586
0,Shoulderstand-Salamba-Sarvangasana-Ekhart-Yoga...,172.0,230.0,0.635740,179.0,233.0,0.623362,178.0,232.0,0.563808,...,0.719426,138.0,114.0,0.765035,140.0,60.0,0.769186,140.0,61.0,0.658067
1,Shoulderstand-Salamba-Sarvangasana-Ekhart-Yoga...,118.0,230.0,0.687847,111.0,233.0,0.691044,111.0,233.0,0.729205,...,0.693057,153.0,114.0,0.633594,151.0,59.0,0.749094,152.0,58.0,0.760944
2,shoulder-stand (1).jpg,276.0,586.0,0.609397,257.0,592.0,0.549326,256.0,594.0,0.642715,...,0.833290,321.0,278.0,0.577032,307.0,117.0,0.692020,305.0,114.0,0.842591
3,shoulder-stand (1)_flipped.jpg,482.0,584.0,0.651710,501.0,594.0,0.558164,500.0,591.0,0.709142,...,0.711145,423.0,262.0,0.864237,452.0,112.0,0.737575,448.0,116.0,0.679890
4,shoulderstand-elisha_flipped.jpg,310.0,537.0,0.358923,317.0,543.0,0.607805,316.0,537.0,0.490563,...,0.421302,247.0,359.0,0.269761,251.0,211.0,0.292691,254.0,215.0,0.203622


In [61]:
sholData.size

364

In [62]:
sholData.shape


(7, 52)

In [64]:
triData=pd.read_csv('csv_per_pose/traingle.csv')
triData.head()

,10-Yoga-poses-for-the-Beginner_4_news4social-300x300.jpg,332.0,204.0,0.74338835,342.0,210.0,0.877148,338.0,204.0.1,0.7380976,...,0.8241222,150.0,340.0.1,0.8660201,322.0,414.0,0.9036975,116.0,415.0,0.8638346
0,10-Yoga-poses-for-the-Beginner_4_news4social-3...,160.0,204.0,0.782980,154.0,204.0,0.841788,151.0,210.0,0.888175,...,0.809379,229.0,355.0,0.924902,376.0,412.0,0.762590,177.0,411.0,0.757497
1,Utthita-Trikonasana-Extended-Triangle-Pose-120...,825.0,430.0,0.810745,847.0,456.0,0.623488,850.0,439.0,0.777007,...,0.795284,316.0,660.0,0.947680,704.0,823.0,0.376546,189.0,818.0,0.890256
2,Utthita-Trikonasana-Extended-Triangle-Pose-120...,379.0,427.0,0.713018,358.0,437.0,0.814116,358.0,456.0,0.765340,...,0.939008,542.0,686.0,0.862105,1012.0,814.0,0.905180,359.0,834.0,0.909712
3,Woman-Doing-Revolved-Triangle.jpg,692.0,569.0,0.830443,665.0,576.0,0.817725,655.0,605.0,0.756738,...,0.829913,1476.0,1021.0,0.842216,878.0,1247.0,0.687239,1684.0,1246.0,0.870313
4,Woman-Doing-Revolved-Triangle_flipped.jpg,1706.0,578.0,0.843500,1742.0,607.0,0.785962,1737.0,576.0,0.790302,...,0.943350,1351.0,1026.0,0.781163,711.0,1245.0,0.905836,1505.0,1251.0,0.665433


In [65]:
triData.size

468

In [66]:
triData.shape

(9, 52)

In [67]:
treeData=pd.read_csv('csv_per_pose/traingle.csv')
treeData.head()

,10-Yoga-poses-for-the-Beginner_4_news4social-300x300.jpg,332.0,204.0,0.74338835,342.0,210.0,0.877148,338.0,204.0.1,0.7380976,...,0.8241222,150.0,340.0.1,0.8660201,322.0,414.0,0.9036975,116.0,415.0,0.8638346
0,10-Yoga-poses-for-the-Beginner_4_news4social-3...,160.0,204.0,0.782980,154.0,204.0,0.841788,151.0,210.0,0.888175,...,0.809379,229.0,355.0,0.924902,376.0,412.0,0.762590,177.0,411.0,0.757497
1,Utthita-Trikonasana-Extended-Triangle-Pose-120...,825.0,430.0,0.810745,847.0,456.0,0.623488,850.0,439.0,0.777007,...,0.795284,316.0,660.0,0.947680,704.0,823.0,0.376546,189.0,818.0,0.890256
2,Utthita-Trikonasana-Extended-Triangle-Pose-120...,379.0,427.0,0.713018,358.0,437.0,0.814116,358.0,456.0,0.765340,...,0.939008,542.0,686.0,0.862105,1012.0,814.0,0.905180,359.0,834.0,0.909712
3,Woman-Doing-Revolved-Triangle.jpg,692.0,569.0,0.830443,665.0,576.0,0.817725,655.0,605.0,0.756738,...,0.829913,1476.0,1021.0,0.842216,878.0,1247.0,0.687239,1684.0,1246.0,0.870313
4,Woman-Doing-Revolved-Triangle_flipped.jpg,1706.0,578.0,0.843500,1742.0,607.0,0.785962,1737.0,576.0,0.790302,...,0.943350,1351.0,1026.0,0.781163,711.0,1245.0,0.905836,1505.0,1251.0,0.665433


In [68]:
treeData.size

468

In [69]:
treeData.shape

(9, 52)

In [70]:
warData=pd.read_csv('csv_per_pose/warrior.csv')
warData.head()

,guy3_warrior045_flipped.jpg,127.0,158.0,0.18773296,130.0,155.0,0.32430094,123.0,153.0,0.2969187,...,0.16830626,141.0,177.0,0.22779596,169.0,83.0,0.16232803,153.0.1,237.0,0.3517379
0,guy3_warrior048_flipped.jpg,130.0,160.0,0.260467,134.0,156.0,0.309977,127.0,156.0,0.325124,...,0.316643,143.0,178.0,0.261489,168.0,84.0,0.326075,152.0,236.0,0.256267
1,guy3_warrior049.jpg,169.0,168.0,0.442089,172.0,164.0,0.508642,166.0,164.0,0.477212,...,0.314839,152.0,190.0,0.133221,153.0,202.0,0.142886,129.0,86.0,0.152305
2,guy3_warrior050_flipped.jpg,131.0,163.0,0.659709,134.0,161.0,0.560365,127.0,159.0,0.657250,...,0.161828,136.0,166.0,0.175403,147.0,188.0,0.107102,129.0,112.0,0.302167
3,guy3_warrior052_flipped.jpg,133.0,165.0,0.477764,137.0,163.0,0.428420,129.0,161.0,0.620494,...,0.290253,131.0,118.0,0.197113,165.0,88.0,0.323873,126.0,112.0,0.347776
4,guy3_warrior055_flipped.jpg,135.0,168.0,0.500988,139.0,166.0,0.533334,132.0,164.0,0.590517,...,0.171324,117.0,118.0,0.154787,162.0,92.0,0.519616,128.0,116.0,0.520726


In [71]:
warData.size

7176

In [72]:
warData.shape

(138, 52)

In [29]:

processed_X_train = preprocess_data(X_train)
processed_X_val =  preprocess_data(X_val)
processed_X_test = preprocess_data(X_test)

In [30]:
inputs = tf.keras.Input(shape=(34))
layer = keras.layers.Dense(128, activation=tf.nn.relu6)(inputs)
layer = keras.layers.Dropout(0.5)(layer)
layer = keras.layers.Dense(64, activation=tf.nn.relu6)(layer)
layer = keras.layers.Dropout(0.5)(layer)
outputs = keras.layers.Dense(len(class_names), activation="softmax")(layer)

In [31]:
model = keras.Model(inputs, outputs)

In [32]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [34]:
checkpoint_path = "weights.best.hdf5"
checkpoint = keras.callbacks.ModelCheckpoint(checkpoint_path,
                             monitor='val_accuracy',
                             verbose=1,
                             save_best_only=True,
                             mode='max')
earlystopping = keras.callbacks.EarlyStopping(monitor='val_accuracy', 
                                              patience=20)


In [35]:
print('--------------TRAINING----------------')
history = model.fit(processed_X_train, y_train,
                    epochs=200,
                    batch_size=16,
                    validation_data=(processed_X_val, y_val),
                    callbacks=[checkpoint, earlystopping])

--------------TRAINING----------------
Epoch 1/200
77/81 [===========================>..] - ETA: 0s - loss: 1.6730 - accuracy: 0.4123
Epoch 1: val_accuracy improved from -inf to 0.54185, saving model to weights.best.hdf5
81/81 [==============================] - 1s 7ms/step - loss: 1.6581 - accuracy: 0.4179 - val_loss: 1.1776 - val_accuracy: 0.5419
Epoch 2/200
62/81 [=====================>........] - ETA: 0s - loss: 1.1343 - accuracy: 0.5585
Epoch 2: val_accuracy improved from 0.54185 to 0.70485, saving model to weights.best.hdf5
81/81 [==============================] - 0s 3ms/step - loss: 1.1101 - accuracy: 0.5704 - val_loss: 0.8552 - val_accuracy: 0.7048
Epoch 3/200
68/81 [========================>.....] - ETA: 0s - loss: 0.9122 - accuracy: 0.6700
Epoch 3: val_accuracy improved from 0.70485 to 0.74449, saving model to weights.best.hdf5
81/81 [==============================] - 0s 3ms/step - loss: 0.9090 - accuracy: 0.6661 - val_loss: 0.6812 - val_accuracy: 0.7445
Epoch 4/200
52/81 [===

Epoch 29/200
59/81 [====================>.........] - ETA: 0s - loss: 0.1698 - accuracy: 0.9449
Epoch 29: val_accuracy did not improve from 0.98678
81/81 [==============================] - 0s 2ms/step - loss: 0.1704 - accuracy: 0.9486 - val_loss: 0.0817 - val_accuracy: 0.9780
Epoch 30/200
77/81 [===========================>..] - ETA: 0s - loss: 0.1788 - accuracy: 0.9359
Epoch 30: val_accuracy did not improve from 0.98678
81/81 [==============================] - 0s 3ms/step - loss: 0.1799 - accuracy: 0.9377 - val_loss: 0.0774 - val_accuracy: 0.9868
Epoch 31/200
63/81 [======================>.......] - ETA: 0s - loss: 0.1624 - accuracy: 0.9444
Epoch 31: val_accuracy did not improve from 0.98678
81/81 [==============================] - 0s 3ms/step - loss: 0.1610 - accuracy: 0.9479 - val_loss: 0.0799 - val_accuracy: 0.9868
Epoch 32/200
61/81 [=====================>........] - ETA: 0s - loss: 0.1545 - accuracy: 0.9508
Epoch 32: val_accuracy did not improve from 0.98678
81/81 [==============

75/81 [==========================>...] - ETA: 0s - loss: 0.0915 - accuracy: 0.9683
Epoch 58: val_accuracy did not improve from 0.99559
81/81 [==============================] - 0s 4ms/step - loss: 0.0880 - accuracy: 0.9704 - val_loss: 0.0375 - val_accuracy: 0.9912
Epoch 59/200
79/81 [============================>.] - ETA: 0s - loss: 0.0820 - accuracy: 0.9707
Epoch 59: val_accuracy did not improve from 0.99559
81/81 [==============================] - 0s 3ms/step - loss: 0.0813 - accuracy: 0.9704 - val_loss: 0.0299 - val_accuracy: 0.9868
Epoch 60/200
80/81 [============================>.] - ETA: 0s - loss: 0.0710 - accuracy: 0.9820
Epoch 60: val_accuracy did not improve from 0.99559
81/81 [==============================] - 0s 3ms/step - loss: 0.0707 - accuracy: 0.9821 - val_loss: 0.0311 - val_accuracy: 0.9912
Epoch 61/200
62/81 [=====================>........] - ETA: 0s - loss: 0.0682 - accuracy: 0.9768
Epoch 61: val_accuracy did not improve from 0.99559
81/81 [===========================

In [36]:
print('-----------------EVAUATION----------------')
loss, accuracy = model.evaluate(processed_X_test, y_test)
print('LOSS: ', loss)
print("ACCURACY: ", accuracy)

-----------------EVAUATION----------------
28/28 [==============================] - 0s 3ms/step - loss: 0.0066 - accuracy: 0.9989
LOSS:  0.006611192133277655
ACCURACY:  0.9988597631454468


In [37]:
tfjs.converters.save_keras_model(model, tfjs_model_dir)
print('tfjs model saved at ',tfjs_model_dir)

tfjs model saved at  model
